### 定义一个ResNetGenerator类

In [ ]:
import torch
import torch.nn as nn
torch.cuda.is_available()

In [ ]:
dir(nn.Module)

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv_block = self.build_conv_block(dim)  # 传入通道数dim，注意此模块的输入输出通道数相同
    
    def build_conv_block(self, dim):
        conv_block = []
        # 添加一个2维反射填充层，反射层宽度为1。关于反射填充层的具体知识可参考：https://blog.csdn.net/qq_43665602/article/details/126593617
        conv_block += [nn.ReflectionPad2d(1)]
        # 添加其他的模块，包括2维卷积层Conv2d、实例归一化层InstanceNorm2d、激活层ReLU
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=True), # bias表示在卷积层中使用【偏置项】，即在每次卷积计算后加一个常量b
                       nn.InstanceNorm2d(dim),
                       nn.ReLU(True)]
        
        conv_block += [nn.ReflectionPad2d(1)]

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=True),
                       nn.InstanceNorm2d(dim)]
        
        # *conv_block代表拆包（unpacking），将【列表】中的每个元素作为单独的参数传递给nn.Sequential()
        return nn.Sequential(*conv_block)
    
    def forward(self, x):
        output = x + self.conv_block(x)  # self.conv_block(x)是在__init__()里面的函数返回值
        
        return output

In [ ]:
class ResNetGenerator(nn.Module):

    def __init__(self, input_nc=3, output_nc=3, nfg=64, n_blocks=9):
        assert(n_blocks >= 0)
        super().__init__()

        self.input_nc = input_nc
        self.output_nc = output_nc
        self.nfg = nfg

        model = [nn.ReflectionPad2d(3), 
                 nn.Conv2d(input_nc, nfg, kernel_size=7, padding=0, bias=True),
                 nn.InstanceNorm2d(nfg),
                 nn.ReLU(True)
                 ]
        
        # 确定下采样次数
        n_downsampling = 2
        for i in range(n_downsampling):
            mult = 2**i
            model += [nn.Conv2d(nfg*mult, nfg*mult*2, kernel_size=3, stride=2, padding=1, bias=True),
                      nn.InstanceNorm2d(nfg*mult*2),
                      nn.ReLU(True)]
            
        mult = 2**n_downsampling
        for i in range(n_blocks):
            model += [ResNetBlock(nfg*mult)]   # ResNetBlock模块的输入输出通道数相同

        for i in range(n_downsampling):
            mult = 2**(n_downsampling-i)   # 第一轮mult=4，第二轮mult=2
            # 添加一个反卷积（上采样）层，用于增大图片尺寸（清晰度）
            model += [nn.ConvTranspose2d(nfg*mult, int(nfg*mult/2), # 第二轮这里就是nfg，故输出通道数就是nfg
                                          kernel_size=3, stride=2, padding=1,
                                          output_padding=1, bias=True),
                      nn.InstanceNorm2d(int(nfg*mult/2)),
                      nn.ReLU(True)]
        
        # 添加一个宽度为3的反射填充层
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(nfg, output_nc, kernel_size=7, padding=0)]     # 上一个循环结束，输出通道数为nfg
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)
    
    def forward(self, input):
        self.output = self.model(input)
        return self.output       


In [ ]:
# 定义
netG = ResNetGenerator()

In [ ]:
# 读取预训练权重
model_path = '../data/p1ch2/horse2zebra_0.4.0.pth'
netG.load_state_dict(torch.load(model_path))

In [ ]:
# 进入评估模式
netG.eval()

In [ ]:
# 预览图片
from PIL import Image
img = Image.open("../data/p1ch2/horse.jpg")
img

In [ ]:
# 预处理图片
from torchvision import transforms

preprocess = transforms.Compose([transforms.Resize(256), transforms.ToTensor()])
img_t = preprocess(img)

# 检查此时图片的数据格式，应该是tensor
type(img_t)

In [ ]:
# 增加一个维度，将数据变成批次
batch_in = torch.unsqueeze(img_t, 0)

In [ ]:
# 将数据输入模型
batch_out = netG(batch_in)
# 检查此时的张量尺寸，其中第0个维度是批次
batch_out.shape

In [ ]:
# 去掉第0个维度，将批次改为单个样本
sample_out = torch.squeeze(batch_out, dim=0)
# 检查样本维度
sample_out.shape

In [ ]:
# 将图像进行归一化操作，把原本处于[-1, 1]区间的数值映射到[0, 1]区间中，增加亮度提高视觉效果
sample_out = (sample_out + 1) / 2

In [ ]:
# 定义一个可以将张量转换成图像的对象
transformer_img = transforms.ToPILImage()
out_img = transformer_img(sample_out)
# out_img.save('../data/p1ch2/zebra.jpg')  
out_img